 # Palmer Penguins
 This notebook contains my analysis of the famous Palmer Penguins dataset.

## Background

The origins of the Palmer Penguins data set can be traced back to the work of Dr. Kristen Gorman and the Palmer Station, a research facility operated by the United States Antarctic Program (USAP). The data contained within this set pertains to the characteristics and species of penguins living on the Palmer Archipegalo in the Antarctic. Being open and accessible, this dataset is commonly used by students starting out in the field of data analytics, as it allows them to practice data manipulation using real-life examples.

## Aim of this project
Placeholder


## Data Frames

Before beginning any sort of exploration of the data, we first need to import the essentials.

The first thing we need to import is pandas, an open-source Python library used for data manipulation and analysis. Since this Python library is geared towards tabular data, it is a perfect tool to explore the dataset and visualise it in a logical way.

In [ ]:
import pandas as pd

: 

Secondly, we will need to load and read in the data set itself. Let's take a quick look:

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv")

: 

In [ ]:
# Let's have a look
df

: 

In [ ]:
# Let's take a look at the first row.
df.iloc[0]

: 

In [5]:
# sex of penguins
df['sex']

0        MALE
1      FEMALE
2      FEMALE
3         NaN
4      FEMALE
        ...  
339       NaN
340    FEMALE
341      MALE
342    FEMALE
343      MALE
Name: sex, Length: 344, dtype: object

In [7]:
# Count the number of penguins of each sex
df['sex'].value_counts()

sex
MALE      168
FEMALE    165
Name: count, dtype: int64

In [8]:
# Describe the body mass of the penguins
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


## References

Data originally published in:

Gorman KB, Williams TD, Fraser WR (2014). Ecological sexual dimorphism and environmental variability within a community of Antarctic penguins (genus Pygoscelis). PLoS ONE 9(3):e90081. https://doi.org/10.1371/journal.pone.0090081

### End